In [3]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('../data/seq2seq/evaluation').resolve()

In [2]:
for split in ['dev', 'eval', 'test']:
    df_res = pd.read_csv(DATA_DIR / f'{split}_results.csv')
    df_err = pd.read_csv(DATA_DIR / f'{split}_errors.csv')
    df_err['pmb_id'] = df_err['pmb_id'].apply(lambda x: f'en/{x}')
    df = pd.concat([df_res, df_err])


    print(f'{split}: f1_strict {df.f1.mean():.3} f1_lenient {df.f1_lenient.mean():.3}')

    df.to_csv(f'seq2seq_results_gold_only_{split}.csv', index=False)

dev: f1_strict 0.867 f1_lenient 0.873
eval: f1_strict 0.85 f1_lenient 0.857
test: f1_strict 0.872 f1_lenient 0.879


In [16]:
from synse.sbn import SBNGraph

IDS_DIR = Path('data/splits')

for split in ['dev', 'eval', 'test']:
    ids = set([
        f'en/{i}' for i in  Path(IDS_DIR / f"{split}.txt").read_text().split("\n")
    ])

    df = pd.read_csv(f'seq2seq_results_{split}.csv')
    ids_missing = ids.difference(set(df.pmb_id.tolist()))

    error_records = []
    for _id in ids_missing:
        try:
            SBNGraph().from_path(f'../../data/pmb_dataset/pmb-extracted/pmb-4.0.0/data/en/gold/{_id.replace("en/", "")}/en.drs.sbn')
        except Exception as e:
            error_records.append(
                {
                    'pmb_id': _id, 
                    'raw_sent': Path(f'../../data/pmb_dataset/pmb-extracted/pmb-4.0.0/data/en/gold/{_id.replace("en/", "")}/en.raw').read_text(),
                    "error": f'Dataset error: {str(e)}'
                }
            )
            
    df_final = pd.concat([df, pd.DataFrame().from_records(error_records)])

    df_final.loc[df_final['raw_sent'].isnull(), 'raw_sent'] = df_final[df_final['raw_sent'].isnull()]['pmb_id'].apply(lambda x: Path(f'../../data/pmb_dataset/pmb-extracted/pmb-4.0.0/data/en/gold/{x.replace("en/", "")}/en.raw').read_text())

    df_final['raw_sent'] = df_final['raw_sent'].str.replace('\n', ' ')
    df_final['raw_sent'] = df_final['raw_sent'].str.rstrip()
    df_final.to_csv(f'seq/seq2seq_results_{split}.csv', index=False)

{'en/p20/d2343', 'en/p10/d1823', 'en/p00/d2229', 'en/p10/d3022', 'en/p40/d2979', 'en/p30/d1528', 'en/p10/d2815'}
{'en/p70/d2267', 'en/p60/d0784'}
{'en/p21/d3487', 'en/p81/d2490', 'en/p01/d1840', 'en/p81/d2197'}


In [3]:
SBNGraph().from_path(f'../../data/pmb_dataset/pmb-extracted/pmb-4.0.0/data/en/gold/p10/d3022/en.drs.sbn')

Initialized cyclic SBN graph, this will work for most tasks, but can cause problems later on when exporting to Penman for instance.


In [31]:
for split in ['dev', 'eval', 'test']:
    df = pd.read_csv(f'seq/seq2seq_results_{split}.csv')

    df['error'] = df['error'].fillna('')
    df = df[~df['error'].str.contains('Dataset error')]

    df['f1'] = df['f1'].fillna(0)
    df['f1_lenient'] = df['f1_lenient'].fillna(0)
    print(f'{split}: f1_strict {df.f1.mean():.3} f1_lenient {df.f1_lenient.mean():.3}')


dev: f1_strict 0.925 f1_lenient 0.933
eval: f1_strict 0.917 f1_lenient 0.926
test: f1_strict 0.924 f1_lenient 0.931


In [32]:
df = pd.read_csv(f'data/results/dev/dev_results_negation.csv')
print(f'{split}: f1_strict {df.f1.mean():.3} f1_lenient {df.f1_lenient.mean():.3}')

test: f1_strict 0.767 f1_lenient 0.779


In [38]:
df = pd.read_csv(f'../data/seq2seq/evaluation/seq2seq_results_gold_only_eval.csv')

df['f1'] = df['f1'].fillna(0)
df['f1_lenient'] = df['f1_lenient'].fillna(0)

df['error'] = df.error.map(lambda i: str(i))

df = df[~df['error'].str.contains("cyclic SBN ")]

In [39]:
df.error

0                                                    nan
1                                                    nan
2                                                    nan
3                                                    nan
4                                                    nan
                             ...                        
823    Invalid token found '-1' in line: election.n.0...
824    Invalid token found '"Leipzig"' in line: sweat...
825    Invalid token found '-2' in line: ranger.n.02 ...
826                                  pop from empty list
827    Invalid token found '+' in line: red.a.01 + Co...
Name: error, Length: 821, dtype: object

In [40]:
df.head()

,pmb_id,raw_sent,precision,recall,f1,precision_lenient,recall_lenient,f1_lenient,error
0,en/p60/d3549,The researchers trained monkeys to play a vide...,0.777778,0.760870,0.769231,0.763158,0.743590,0.753247,nan
1,en/p60/d2338,This tool is good for nothing.,0.952381,0.952381,0.952381,0.944444,0.944444,0.944444,nan
2,en/p60/d1579,Tom is puzzled.,0.913043,0.913043,0.913043,0.950000,0.950000,0.950000,nan
3,en/p60/d3402,This is an important film.,0.939394,0.939394,0.939394,0.928571,0.928571,0.928571,nan
4,en/p60/d3280,She wrapped her sandwich in plastic wrap.,0.882353,0.750000,0.810811,0.862069,0.735294,0.793651,nan


In [41]:
print(len(df[df.error != 'nan']))
print(f'f1_strict {df.f1.mean():.3} f1_lenient {df.f1_lenient.mean():.3}')

25
f1_strict 0.824 f1_lenient 0.831


In [37]:
len(df[df.error != 'nan'].error)

18

In [44]:
(18 / 832) * 100

2.1634615384615383